# 2 Batch data analysis
>In this problem, we will look into batch data; dynamic time-series of a finite duration. Batch manufacturing processes are very common in chemical, pharma, bioengineering and semiconductor industries such as baker's yeast production, beer brewing and vaccines production.


>In theory, a reactor is designed with temperature, pressure, level, pH control and multiple sensors that measure these variables among others. A perfect batch (again in theory) is one that is tightly controlled to the specications and as a result the productivity and quality of
the final product is optimized.


>In real life, a typical batch is run from a few hours up to a week or two and a lot of things can go wrong during this period. There is always variability either because the process is
very sensitive to minor 
fluctuations in some variables or the control of some variables failed
for a period of time.


>In a company that implements Data Analytics or Multivariate Statistical Process Control (MSPC) monitoring is typically implemented with the following steps:


> 1. Identify a number of reference, perfect **historical** batches (15-20), both in terms of high
productivity/quality and minimum anomalies or 
uctuations around the setpoints.

> 2. Create a PCA model of the perfect batches identified. This is your **model**.

> 3. Every time your site is running a new batch, **fit** your data **online** or as soon as your data infrastructure allows you to do so. Fitting will tell you whether your batch is similar to the perfect batches or it is deviating from the reference behaviour.

>Next, you will follow these steps to build a Batch Statistical Process Control and implement it to monitor a new batch (we will assume that you got the data at the end of the batch and t them to the model). The dataset is from a baker's yeast production facility in Solna, Sweden capturing the last step of the fermentation.

## 2.1 Build a Batch Statistical Process Control model
> 1. Import the data from 'bakers yeast reference batches.xlsx'. Identify how many batches are in the data. What is the duration of each batch and how many data points are there per batch? How many variables are measured (including time)?

> 2. Plot the variables time-proles in a 2x4 subplot. Inspect the graphs (don't just plot them). Look for potential outliers. Which variables have the largest variability? Which variables are tightly controlled?

> 3. Select the features (including the Time column), pre-process the data and perform PCA with 5 principal components. Extract the scores and loadings.

> 4. In order to plot the scores-loading plot, you need to pivot the scores BatchID with index 'Time' (use pandas pivot table).

> 5. Plot the scores-loadings plot with one line per batch (this is why the pivot in the previous step was needed). The output should look similar to the plot below. You may choose a dierent scaling, but the trend should be the same as this graph.

> 6. Explain this graph. In which quarter do the batches start and end? What happens at the kink where the direction of the lines changes? Can you tell from this graph which variables do not change in the rst phase and which in the second phase?

> 7. Plot the cumulative explained variance. How much variance do the first two principal components capture?

##2.2 Use the model to monitor running batches
>The goal of building an unsupervised model is to monitor the running batches. Your site runs two reactors in parallel and here you will fit the data from these two reactors to the model previously built and identify potential problems and outliers.


> 1. Load the data from the file 'todays batches.xlsx' and repeat the same procedure as in the steps 3-4 of the previous section with the exception of the PCA modeling. Here, instead of fit the data to the model and transform, you will only transform them with the model object you created in the previous section.

> 2. Plot the same scores-loadings plot for the data in the batches you used to develop the model with solid lines. Overlay the new incoming data from the two current batches with dashed lines and two different colours to distinguish them. Also, add a legend for the two batches so that the viewer can distinguish them.

> 3. Do the batches show behaviour similar to that of the reference ones or there are outliers indicating potential problems?